In [35]:
%%capture --no-stderr
!pip install langchain langchain-openai langchain-core langchain-community
!pip install langsmith langchain-experimental

In [103]:
import os
import tqdm
import json
import uuid
import copy
import logging
from enum import Enum
from google.colab import userdata, drive
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from pydantic import BaseModel, Field, field_validator

In [39]:
logging.getLogger("langchain_experimental.utilities.python").setLevel(logging.ERROR)

In [54]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
os.environ["LANGCHAIN_PROJECT"] = "exercises-creation-langchain"

In [5]:
llm = ChatOpenAI(model="gpt-4.1-nano")

In [15]:
exercises_content = """
- tipos de dados
- condicionais
- laços de repetição (while e for)
- Arrays NumPy e operações vetorizadas
- Estruturas de dados com pandas: Series e DataFrame
- Indexação, filtros e operações básicas com colunas
"""

In [7]:
exercise_topics_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é ajudar a criação de
exercícios para avaliar os alunos de uma disciplina EAD.

Sua tarefa nesse momento é gerar os tópicos que serão usados para a criação dos
exercícios. Para cada tópico será gerado um exercício correspondente.

Gere um total de {number_of_exercises} tópicos

Considere o seguinte conteúdo que deve ser abordado nos exercícios
{exercises_content}
"""

In [8]:
class Topic(BaseModel):
    text: str = Field(description='content of topic text string')

In [9]:
class ExercisesContent(BaseModel):
    topics: list[Topic] = Field(description="list of topics that will be used to create the exercises")

In [13]:
exercise_topics_chain = (
    ChatPromptTemplate.from_template(exercise_topics_prompt)
    | llm.with_structured_output(ExercisesContent)
)

In [16]:
exercise_topics_response = exercise_topics_chain.invoke(
    {
        "number_of_exercises": 10,
        "exercises_content": exercises_content
    }
)

In [19]:
exercise_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é ajudar a criação de
exercícios para avaliar os alunos de uma disciplina EAD.

Sua tarefa nesse momento é gerar um exercício de fixação prático, ou seja, a questão deve pedir para
o aluno deverá escrever o código em Python para sua resolução.

O nível do exercício deve ser {exercise_level}

Gere também a resposta em Python para ser usada como gabarito.

Considere o seguinte tópico que deve ser abordado no exercício:
{topic}
"""

In [20]:
class Exercise(BaseModel):
    question: str = Field(description="question statement")
    code_answer: str = Field(description="python code that answer the question")

In [21]:
exercise_chain = (
    ChatPromptTemplate.from_template(exercise_prompt)
    | llm.with_structured_output(Exercise)
)

In [24]:
exercise = exercise_chain.invoke(
    {
        "exercise_level": "médio",
        "topic": exercise_topics_response.topics[4].text
    }
)

In [43]:
def has_code_error(output: str) -> bool:
    return "Traceback" in output or "Error" in output

In [50]:
%%time
exercises = []
for topic in tqdm.tqdm(exercise_topics_response.topics):
    exercise = exercise_chain.invoke(
        {
            "exercise_level": "médio",
            "topic": topic
        }
    )
    exercises.append(exercise)

100%|██████████| 10/10 [00:17<00:00,  1.75s/it]

CPU times: user 447 ms, sys: 50.1 ms, total: 497 ms
Wall time: 17.5 s


##**Criação do Notebook**

In [112]:
titles = []
subtopic_contents = []
cells = []
for idx, exercise in enumerate(exercises):
    cells.append(
        {
        "cell_type": "markdown",
        "source":f"## **Exercício {idx + 1}**\n\n{exercise.question}",
        "metadata": {
            "id": ""
        }
    }

    )
    cells.append(
        {
            "cell_type": "code",
            "execution_count": None,
            "metadata": {
                "id": ""
            },
            "outputs": [],
            "source": exercise.code_answer
        }
    )

In [64]:
path = '/content/drive/MyDrive/01 - PUC/2025-01/Python para DS/listas'

In [113]:
notebook_gab = {
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": cells
}

In [69]:
summary_prompt = """
#################### Contexto ####################

Você é um assistente para a geração de conteúdo, seu papel é ajudar a criação de
exercícios para avaliar os alunos de uma disciplina EAD.

Sua tarefa é gerar uma descrição do notebook de forma resumida e criar um título
para o notebook.

Use o seguinte conteúdo para criar a descrição e o título:
{content}
"""

In [70]:
class NotebookMarkdownHeader(BaseModel):
    title: str = Field(description="notebook title")
    description: str = Field(description="notebook description")

In [71]:
header_chain = (
    ChatPromptTemplate.from_template(summary_prompt)
    | llm.with_structured_output(NotebookMarkdownHeader)
)

In [72]:
header_response = header_chain.invoke(
    {
        "content": notebook_gab
    }
)

In [114]:
header_cell = {
        "cell_type": "markdown",
        "source":f"""# **{header_response.title}**\n\n**Autor**: Renan Santos Mendes\n\n**Email**: renansantosmendes@gmail.com\n\n**Descrição**: {header_response.description}""",
        "metadata": {
            "id": ""
        }
    }

In [115]:
notebook_gab.get('cells').insert(0, header_cell)

In [94]:
notebook_id = uuid.uuid4().hex

with open(
    file=os.path.join(path, f"notebook_{notebook_id}_gab.ipynb"),
    mode="w",
    encoding="utf-8"
    ) as f:

    json.dump(notebook_gab, f, indent=4, ensure_ascii=False)

In [116]:
notebook = copy.deepcopy(notebook_gab)

In [118]:
for cell in notebook.get('cells'):
    if cell.get('cell_type') == 'code' and cell.get('source'):
        cell['source']= ""

In [121]:
with open(
    file=os.path.join(path, f"notebook_{notebook_id}.ipynb"),
    mode="w",
    encoding="utf-8"
    ) as f:

    json.dump(notebook, f, indent=4, ensure_ascii=False)